In [ ]:
!pip install tensorflow-gpu==2.8.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 67.2 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: tensorflow-gpu
    Found existing installation: tensorflow-gpu 2.8.0rc0
    Uninstalling tensorflow-gpu-2.8.0rc0:
      Successfully uninstalled tensorflow-gpu-2.8.0rc0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.8.0 which is incompatible.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.8.0 which is incompatible.


In [ ]:
#Now importing modules
import helper
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Embedding, SimpleRNN
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy


In [ ]:
import requests

# Define the paths
english_path = 'https://raw.githubusercontent.com/projjal1/datasets/master/small_vocab_en.txt'
french_path = 'https://raw.githubusercontent.com/projjal1/datasets/master/small_vocab_fr.txt'

# Function to download data from URL
def download_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to download data from {url}")
        return None

# Download English data
english_data = download_data(english_path)

# Download French data
french_data = download_data(french_path)

# Check if data is downloaded successfully
if english_data and french_data:
    # Process the data further as per your requirement
    # For example, you can split the text into lines, tokenize it, etc.
    english_sentences = english_data.split('\n')
    french_sentences = french_data.split('\n')

    # Print sample sentences
    print("Sample English Sentence:", english_sentences[0])
    print("Sample French Sentence:", french_sentences[0])
else:
    print("Data download failed. Please check the URLs.")


Sample English Sentence: new jersey is sometimes quiet during autumn , and it is snowy in april .
Sample French Sentence: new jersey est parfois calme pendant l' automne , et il est neigeux en avril .


In [ ]:
import os

def load_data(path):
  input_file = os.path.join(path)
  with open(input_file, "r") as f:
    data = f.read()

  return data.split('\n')

In [ ]:
import tensorflow as tf

# Define the paths
english_path = 'https://raw.githubusercontent.com/projjal1/datasets/master/small_vocab_en.txt'
french_path = 'https://raw.githubusercontent.com/projjal1/datasets/master/small_vocab_fr.txt'

# Using tf.keras.utils.get_file to import dataset
english_data = tf.keras.utils.get_file('file1', english_path)
french_data = tf.keras.utils.get_file('file2', french_path)


10149888/10135741 [==============================] - 0s 0us/step


In [ ]:
#Now loading data
english_sentences=load_data(english_data)
french_sentences=load_data(french_data)

In [ ]:
for i in range(5):
  print('Sample :',i)
  print(english_sentences[i])
  print(french_sentences[i])
  print('-'*50)

Sample : 0
new jersey is sometimes quiet during autumn , and it is snowy in april .
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
--------------------------------------------------
Sample : 1
the united states is usually chilly during july , and it is usually freezing in november .
les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
--------------------------------------------------
Sample : 2
california is usually quiet during march , and it is usually hot in june .
california est généralement calme en mars , et il est généralement chaud en juin .
--------------------------------------------------
Sample : 3
the united states is sometimes mild during june , and it is cold in september .
les états-unis est parfois légère en juin , et il fait froid en septembre .
--------------------------------------------------
Sample : 4
your least liked fruit is the grape , but my least liked is the apple .
votre moins aimé frui

In [ ]:
import collections

In [ ]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('English Vocab:',len(english_words_counter))
print('French Vocab:',len(french_words_counter))

English Vocab: 227
French Vocab: 355


In [ ]:
def tokenize(x):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(x)
  return tokenizer.texts_to_sequences(x), tokenizer

In [ ]:

# Tokenize Sample output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']

text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()

for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
  print('Sequence {} in x'.format(sample_i + 1))
  print('  Input:  {}'.format(sent))
  print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [ ]:
def pad(x, length=None):
  return pad_sequences(x, maxlen=length, padding='post')

In [ ]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    #Expanding dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [ ]:
def logits_to_text(logits, tokenizer):
  index_to_words = {id: word for word, id in tokenizer.word_index.items()}
  index_to_words[0] = ''

  #So basically we are predicting output for a given word and then selecting best answer
  #Then selecting that label we reverse-enumerate the word from id
  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])


In [ ]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.005

    # TODO: Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(GRU(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


In [ ]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

In [ ]:
simple_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

In [ ]:
simple_rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 21, 256)           51200     
                                                                 
 gru (GRU)                   (None, 21, 256)           394752    
                                                                 
 time_distributed (TimeDistr  (None, 21, 1024)         263168    
 ibuted)                                                         
                                                                 
 dropout (Dropout)           (None, 21, 1024)          0         
                                                                 
 time_distributed_1 (TimeDis  (None, 21, 345)          353625    
 tributed)                                                       
                                                                 
Total params: 1,062,745
Trainable params: 1,062,745
Non-

In [ ]:
history=simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)


Epoch 1/20
108/108 [==============================] - 441s 4s/step - loss: 1.3793 - accuracy: 0.6783 - val_loss: 0.4801 - val_accuracy: 0.8466
Epoch 2/20
108/108 [==============================] - 433s 4s/step - loss: 0.4119 - accuracy: 0.8641 - val_loss: 0.3043 - val_accuracy: 0.8986
Epoch 3/20
108/108 [==============================] - 419s 4s/step - loss: 0.2947 - accuracy: 0.9011 - val_loss: 0.2411 - val_accuracy: 0.9175
Epoch 4/20
108/108 [==============================] - 428s 4s/step - loss: 0.2480 - accuracy: 0.9159 - val_loss: 0.2219 - val_accuracy: 0.9241
Epoch 5/20
108/108 [==============================] - 429s 4s/step - loss: 0.2226 - accuracy: 0.9237 - val_loss: 0.2054 - val_accuracy: 0.9293
Epoch 6/20
108/108 [==============================] - 420s 4s/step - loss: 0.2068 - accuracy: 0.9282 - val_loss: 0.1971 - val_accuracy: 0.9319
Epoch 7/20
108/108 [==============================] - 428s 4s/step - loss: 0.1971 - accuracy: 0.9311 - val_loss: 0.1918 - val_accuracy: 0.9336

In [ ]:
simple_rnn_model.save('model.h5')

In [ ]:
def final_predictions(text):
  y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
  y_id_to_word[0] = ''

  sentence = [english_tokenizer.word_index[word] for word in text.split()]
  sentence = pad_sequences([sentence], maxlen=preproc_french_sentences.shape[-2], padding='post')

  print(sentence.shape)
  print(logits_to_text(simple_rnn_model.predict(sentence[:1])[0], french_tokenizer))

In [ ]:
import re

In [ ]:
txt=input().lower()

final_predictions(re.sub(r'[^\w]', ' ', txt))

The weather is nice and warm
(1, 21)
les temps est agréable agréable chaud               
